In [12]:
import csv
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

"""
protests = pd.read_csv('Public view RUS.csv')
events = protests[protests['event_type'] == 'собрание']
events = events[events['subject_type'] != 'политический протест']
events = events[events['region'] != 'Москва']
"""

#inputdata2 = pd.read_csv('datasets/superdataset-24-alltime-clust (IQR)-normbysoul-f (conflict, no output).csv')
inputdata = pd.read_csv('/content/result (3).csv')
output = pd.read_excel('/content/Conflict assessment (top300) 21 neworder formodel (1).xlsx')

#output = output[output['sum'] != 0]


output = output.sort_values(by=['oktmo', 'year'])

output['Экологический'] = output[['1', '2', '3', '4', '5', '6']].sum(axis=1)
output['Этнический'] = output[['7', '8', '9','10', '11', '12']].sum(axis=1)
output['Промышленный'] = output[['13', '14', '15', '16', '17', '18', '19', '20', '21']].sum(axis=1)


output['Экологический'] = output['Экологический'] / output['sum'].max()
output['Этнический'] = output['Этнический'] / output['sum'].max()
output['Промышленный'] = output['Промышленный'] / output['sum'].max()

# преобразование социального риска в шкалу от 0 до 1
output['sum'] = output['sum'] / output['sum'].max()
#inputdata = final

# Convert 'year' and 'oktmo' to float in both dataframes before merging
inputdata['year'] = inputdata['year'].astype(float)
inputdata['oktmo'] = inputdata['oktmo'].astype(float)
output['year'] = output['year'].astype(float)
output['oktmo'] = output['oktmo'].astype(float)
output.to_csv('Risk_for_illustration.csv', index=False)

# Merge inputdata with relevant output columns on 'oktmo' and 'year'
inputdata = pd.merge(inputdata, output[['oktmo', 'year', 'sum']], on=['oktmo', 'year'], how='left')

# Rename the 'sum' column from output to 'risk'
inputdata = inputdata.rename(columns={'sum': 'risk'})


# подготовка входного и выходного результата для модели
# совмещение факторов теущего года с социальным риском следующего
examples = []
for i in range(len(inputdata) - 1):
    if inputdata.iloc[i]['oktmo'] == inputdata.iloc[i + 1]['oktmo']:
        if inputdata.iloc[i]['year'] + 1 == inputdata.iloc[i + 1]['year']:
            #inputdata.iloc[i, inputdata.shape[1] - 1] = inputdata.iloc[i + 1, inputdata.shape[1] - 1]
            inputdata.loc[i, 'risk'] = inputdata.loc[i + 1, 'risk']
            examples.append(inputdata.iloc[i])


examples = np.array(examples)
features = inputdata.columns
examples = pd.DataFrame(examples, columns=features)

#examples = examples[examples['risk'] != 0]

#examples = examples.drop(columns=['oktmo', 'year', 'name'])
examples.to_csv('data_for_pred.csv', index=False)

